In [3]:
import cv2 
import os

def recognise(image_path):
    # path(image filename)
    path = image_path
    
    # Reading an image in grayscale mode 
    image = cv2.imread(path, 0) 
    image = cv2.bitwise_not(image)
    # Window name in which image is displayed 
    # window_name = 'image'
    
    # Using cv2.imshow() method 
    # Displaying the image 
    # cv2.imshow(window_name, image) 
    
    # waits for user to press any key 
    # (this is necessary to avoid Python kernel form crashing) 
    # cv2.waitKey(0) 
    
    # closing all open windows 
    # cv2.destroyAllWindows()
    # --------------------------------------------------------------------
    bb_info_list = []

    min_contour_area = 100
    image_with_rectangles = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    image_with_rectangles = image.copy()
    image_with_rectangles = cv2.cvtColor(image_with_rectangles, cv2.COLOR_GRAY2BGR)

    from PIL import Image 
    # get image 
    img = Image.open(path) 
    
    # get width and height 
    width = img.width 
    height = img.height

    for contour in contours:
        contour_area = cv2.contourArea(contour)
        if contour_area >= min_contour_area:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(image_with_rectangles, (x, y), (x + w, y + h), (0, 0, 255), 1)
            [xmin, xmax, ymin, ymax,  img_width, img_height] = x, (x+w), y, (y+h), img.width, img.height 
            bb_info_list.append([xmin, xmax, ymin, ymax,  img_width, img_height])

    # Initialize a list to store the grouped bounding boxes
    grouped_bboxes = []

    # Sort the bounding boxes by their x-minimum value
    bb_info_list.sort(key=lambda bbox: bbox[0])

    # Initialize a temporary list to store the current group
    temp_group = [bb_info_list[0]]

    # Loop through the bounding boxes and check for overlap on the x-axis
    for i in range(1, len(bb_info_list)):
        prev_bbox = temp_group[-1]
        current_bbox = bb_info_list[i]

        # Check for x-axis overlap by comparing xmax of the previous bbox with xmin of the current bbox
        if prev_bbox[1] >= current_bbox[0]:
            # There is an overlap, add the current bbox to the temporary group
            temp_group.append(current_bbox)
        else:
            # No overlap, finalize the current group and start a new one
            grouped_bboxes.append(temp_group)
            temp_group = [current_bbox]

    # After the loop, the last group may not have been added, so add it
    grouped_bboxes.append(temp_group)

    print(grouped_bboxes)
    # Print or use the grouped bounding boxes

    temp = []

    for i in range(len(bb_info_list)):
        combined = False
        current_box = bb_info_list[i]

        for j in range(len(temp)):
            if temp[j][0] <= current_box[0] and temp[j][1] >= current_box[1]:
                # Current box is completely within the x-range of an existing box
                temp[j][2] = min(temp[j][2], current_box[2])
                temp[j][3] = max(temp[j][3], current_box[3])
                combined = True
            elif current_box[0] <= temp[j][0] and current_box[1] >= temp[j][1]:
                # Existing box is completely within the x-range of the current box
                temp[j][0] = min(temp[j][0], current_box[0])
                temp[j][1] = max(temp[j][1], current_box[1])
                combined = True

        if not combined:
            temp.append(current_box)


    #To change pathname to {class}.jpg
    cv2.imwrite('/content/hpboundrec.jpg', image_with_rectangles[ymin:ymax, xmin:xmax])
    # cv2.imshow(window_name,image_with_rectangles)
    # cv2.waitKey(0) 
    # --------------------------------------------------------------------
    temp.sort(key=lambda x: x[1])
    temp
    # --------------------------------------------------------------------
    import os
    import shutil

    #General gist of code chunk: Create a folder with the image filename, save components of image in the folder

    i = 1
    # Folder name you want to create under 'all_equations'
    subfolder_name = "equation" + str(i)

    # Path to 'all_equations' folder
    all_equations_dir = "all_equations"

    # Create the subfolder under 'all_equations' if it doesn't exist
    subfolder_path = os.path.join(all_equations_dir, subfolder_name)

    if not os.path.exists(subfolder_path):
        os.mkdir(subfolder_path)
        print(f"Folder '{subfolder_name}' created under '{all_equations_dir}'.")
    else:
        print(f"Folder '{subfolder_name}' under '{all_equations_dir}' already exists.")

    j = 1

    # Cropping individual images and saving in the single subfolder
    for info in temp:
        object = image[info[2]:info[3], info[0]:info[1]]
        image_path = os.path.join(subfolder_path, f'item_{j}.jpg')
        cv2.imwrite(image_path, object)
        j += 1

    i += 1 
    #--------------------------------------------------------------------------------------------------

folder_path = 'data'  # Replace with the actual folder path

# List all files in the folder
file_list = os.listdir(folder_path)

# Filter the list to include only image files (you can adjust the file extensions)
image_extensions = ['.jpg', '.png', '.jpeg']
image_paths = [os.path.join(folder_path, file) for file in file_list if any(file.lower().endswith(ext) for ext in image_extensions)]

# Iterate through the image paths
for image_path in image_paths:
    recognise(image_path)




[[[75, 180, 190, 349, 1497, 998]], [[225, 313, 247, 332, 1497, 998]], [[373, 491, 214, 350, 1497, 998]], [[533, 626, 269, 279, 1497, 998], [571, 594, 228, 255, 1497, 998], [583, 612, 286, 314, 1497, 998]], [[690, 794, 182, 376, 1497, 998]], [[851, 943, 258, 268, 1497, 998], [897, 923, 282, 309, 1497, 998], [898, 922, 222, 242, 1497, 998]], [[956, 1130, 174, 470, 1497, 998]]]
Folder 'equation1' created under 'all_equations'.


[[[75, 180, 190, 349, 1497, 998]], [[225, 313, 247, 332, 1497, 998]], [[373, 491, 214, 350, 1497, 998]], [[533, 626, 269, 279, 1497, 998], [571, 594, 228, 255, 1497, 998], [583, 612, 286, 314, 1497, 998]], [[690, 794, 182, 376, 1497, 998]], [[851, 943, 258, 268, 1497, 998], [897, 923, 282, 309, 1497, 998], [898, 922, 222, 242, 1497, 998]], [[956, 1130, 174, 470, 1497, 998]]]


97

[[75, 180, 190, 349, 1497, 998],
 [225, 313, 247, 332, 1497, 998],
 [373, 491, 214, 350, 1497, 998],
 [533, 626, 228, 314, 1497, 998],
 [690, 794, 182, 376, 1497, 998],
 [851, 943, 222, 309, 1497, 998],
 [956, 1130, 174, 470, 1497, 998]]

Folder 'equation1' under 'all_equations' already exists.


: 